In [10]:
import pandas as pd
import numpy as np
import pymongo
import json
import sys

In [11]:
uri = 'mongodb://localhost:27017'
client = pymongo.MongoClient(uri)

In [12]:
db = client['ppl_data']
results = db['results']
scores = db['scores']
standings = db['standings']
print 'counts: (results, scores, standings)'
results.estimated_document_count(), scores.estimated_document_count(), standings.estimated_document_count()

counts: (results, scores, standings)


(2729, 31461, 1753)

In [13]:
results.find().limit(1)[0]

{u'_id': {u'group_id': 1, u'season_id': 5, u'week_id': 1},
 u'games': [u'Amer Most Haunt', u'TX Sector', u'Attack From Mars'],
 u'location': u'Pinball Factory',
 u'results': [{u'player': u'Cathy Alioto',
   u'scores': [{u'points': 2, u'score': u'4,732,840'},
    {u'points': 3, u'score': u'263,570'},
    {u'points': 2, u'score': u'511,447,960'}]},
  {u'player': u'Rich Mahathey',
   u'scores': [{u'points': 3, u'score': u'18,881,400'},
    {u'points': 5, u'score': u'358,660'},
    {u'points': 5, u'score': u'3,810,918,840'}]},
  {u'player': u'Al Thomka',
   u'scores': [{u'points': 5, u'score': u'24,716,640'},
    {u'points': 1, u'score': u'disqualified'},
    {u'points': 3, u'score': u'604,411,930'}]},
  {u'player': u'Lauren Kester',
   u'scores': [{u'points': 1, u'score': u'3,642,950'},
    {u'points': 2, u'score': u'179,360'},
    {u'points': 1, u'score': u'142,252,490'}]}]}

In [14]:
scores.find().limit(1)[0]

{u'_id': ObjectId('5ca819aee0e7b26fd082329b'),
 u'date': u'April 9, 2015',
 u'game_id': 1,
 u'game_name': u'Amer Most Haunt',
 u'group_id': 1,
 u'location': u'Pinball Factory',
 u'player': u'Cathy Alioto',
 u'points': 2,
 u'position': 0,
 u'score': u'4,732,840',
 u'season_id': 5,
 u'week_id': 1}

In [15]:
standings.find().limit(1)[0]

{u'_id': {u'player': u'Aleksander Kaczmarczyk', u'season_id': 15},
 u'adjusted_points': 86,
 u'average_points': 13.375,
 u'points': {u'week_1': 15,
  u'week_2': 13,
  u'week_3': 15,
  u'week_4': 10,
  u'week_5': 15,
  u'week_6': 13,
  u'week_7': 15,
  u'week_8': 11},
 u'total_points': 107,
 u'weeks_played': 8}

In [16]:
# season_ids = [5,6,7,8,9,10,11,12,13,14,15,16]
season_ids = [16]

find_query = {'$or': []}
for season_id in season_ids:
    find_query.get('$or').append({'season_id': season_id})
print 'find: ', find_query

df = pd.DataFrame()
i=0
for item in scores.find(find_query, projection={'_id': False}, sort=[
    ('season_id', 1), ('game_id', 1), ('week_id', 1), ('group_id', 1), ('points', -1)
]):
    df = df.append(item, ignore_index=True)
    i=i+1
    if i%500==0:
        print 'Items processed: {}'.format(i)
print 'Items processed: {}'.format(i)
df.head()

find:  {'$or': [{'season_id': 16}]}
Items processed: 500
Items processed: 1000
Items processed: 1500
Items processed: 2000
Items processed: 2163


,date,game_id,game_name,group_id,location,player,points,position,score,season_id,week_id
0,"January 21, 2019",1.0,Attack From Mars,1.0,Dojo,Mark Lawrence,5.0,2.0,"3,220,471,780",16.0,1.0
1,"January 21, 2019",1.0,Attack From Mars,1.0,Dojo,Pete Hendricks,3.0,1.0,"2,360,571,610",16.0,1.0
2,"January 21, 2019",1.0,Attack From Mars,1.0,Dojo,Peace Nick Mann,2.0,0.0,"1,388,813,900",16.0,1.0
3,"January 21, 2019",1.0,Attack From Mars,1.0,Dojo,Sandy Graver,1.0,3.0,"590,859,230",16.0,1.0
4,"January 21, 2019",1.0,Super Orbit,2.0,Dojo,Brian Uplinger,5.0,1.0,"527,230",16.0,1.0


In [17]:
df.head(20)

,date,game_id,game_name,group_id,location,player,points,position,score,season_id,week_id
0,"January 21, 2019",1.0,Attack From Mars,1.0,Dojo,Mark Lawrence,5.0,2.0,"3,220,471,780",16.0,1.0
1,"January 21, 2019",1.0,Attack From Mars,1.0,Dojo,Pete Hendricks,3.0,1.0,"2,360,571,610",16.0,1.0
2,"January 21, 2019",1.0,Attack From Mars,1.0,Dojo,Peace Nick Mann,2.0,0.0,"1,388,813,900",16.0,1.0
3,"January 21, 2019",1.0,Attack From Mars,1.0,Dojo,Sandy Graver,1.0,3.0,"590,859,230",16.0,1.0
4,"January 21, 2019",1.0,Super Orbit,2.0,Dojo,Brian Uplinger,5.0,1.0,"527,230",16.0,1.0
5,"January 21, 2019",1.0,Super Orbit,2.0,Dojo,Kevin Vitale,3.0,3.0,"475,980",16.0,1.0
6,"January 21, 2019",1.0,Super Orbit,2.0,Dojo,Aaron Crittenden,2.0,0.0,"448,040",16.0,1.0
7,"January 21, 2019",1.0,Super Orbit,2.0,Dojo,Virginia Hendricks,1.0,2.0,"366,290",16.0,1.0
8,"January 21, 2019",1.0,TheAddamsFam,3.0,Dojo,Matt Schwab,5.0,0.0,"132,990,050",16.0,1.0
9,"January 21, 2019",1.0,TheAddamsFam,3.0,Dojo,Jason Potter,3.0,2.0,"55,720,200",16.0,1.0


In [18]:
# FIXME: bad performance
def gen_arrays(var):
    var['wins_array'] = array[int(var.name)+1:]
    var['losses_array'] = array[:int(var.name)]
    return var

grouped = df.groupby(['season_id', 'week_id', 'group_id', 'game_id'])
df2 = pd.DataFrame()
for name, group in grouped:
    array = group['player'].as_matrix()[:]
    group.reset_index(inplace=True)
#     group['wins'] = np.flipud(group.index.values)
#     group['losses'] = group.index.values
    print 'Generating win/loss arrays for group: {}'.format(name)
    group = group.apply(gen_arrays, axis=1)
    df2 = df2.append(group)
df2.head(10)

Generating win/loss arrays for group: (16.0, 1.0, 1.0, 1.0)


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.


Generating win/loss arrays for group: (16.0, 1.0, 1.0, 2.0)
Generating win/loss arrays for group: (16.0, 1.0, 1.0, 3.0)
Generating win/loss arrays for group: (16.0, 1.0, 2.0, 1.0)
Generating win/loss arrays for group: (16.0, 1.0, 2.0, 2.0)
Generating win/loss arrays for group: (16.0, 1.0, 2.0, 3.0)
Generating win/loss arrays for group: (16.0, 1.0, 3.0, 1.0)
Generating win/loss arrays for group: (16.0, 1.0, 3.0, 2.0)
Generating win/loss arrays for group: (16.0, 1.0, 3.0, 3.0)
Generating win/loss arrays for group: (16.0, 1.0, 4.0, 1.0)
Generating win/loss arrays for group: (16.0, 1.0, 4.0, 2.0)
Generating win/loss arrays for group: (16.0, 1.0, 4.0, 3.0)
Generating win/loss arrays for group: (16.0, 1.0, 5.0, 1.0)
Generating win/loss arrays for group: (16.0, 1.0, 5.0, 2.0)
Generating win/loss arrays for group: (16.0, 1.0, 5.0, 3.0)
Generating win/loss arrays for group: (16.0, 1.0, 6.0, 1.0)
Generating win/loss arrays for group: (16.0, 1.0, 6.0, 2.0)
Generating win/loss arrays for group: (1

Generating win/loss arrays for group: (16.0, 3.0, 3.0, 3.0)
Generating win/loss arrays for group: (16.0, 3.0, 4.0, 1.0)
Generating win/loss arrays for group: (16.0, 3.0, 4.0, 2.0)
Generating win/loss arrays for group: (16.0, 3.0, 4.0, 3.0)
Generating win/loss arrays for group: (16.0, 3.0, 5.0, 1.0)
Generating win/loss arrays for group: (16.0, 3.0, 5.0, 2.0)
Generating win/loss arrays for group: (16.0, 3.0, 5.0, 3.0)
Generating win/loss arrays for group: (16.0, 3.0, 6.0, 1.0)
Generating win/loss arrays for group: (16.0, 3.0, 6.0, 2.0)
Generating win/loss arrays for group: (16.0, 3.0, 6.0, 3.0)
Generating win/loss arrays for group: (16.0, 3.0, 7.0, 1.0)
Generating win/loss arrays for group: (16.0, 3.0, 7.0, 2.0)
Generating win/loss arrays for group: (16.0, 3.0, 7.0, 3.0)
Generating win/loss arrays for group: (16.0, 3.0, 8.0, 1.0)
Generating win/loss arrays for group: (16.0, 3.0, 8.0, 2.0)
Generating win/loss arrays for group: (16.0, 3.0, 8.0, 3.0)
Generating win/loss arrays for group: (1

Generating win/loss arrays for group: (16.0, 5.0, 3.0, 2.0)
Generating win/loss arrays for group: (16.0, 5.0, 3.0, 3.0)
Generating win/loss arrays for group: (16.0, 5.0, 4.0, 1.0)
Generating win/loss arrays for group: (16.0, 5.0, 4.0, 2.0)
Generating win/loss arrays for group: (16.0, 5.0, 4.0, 3.0)
Generating win/loss arrays for group: (16.0, 5.0, 5.0, 1.0)
Generating win/loss arrays for group: (16.0, 5.0, 5.0, 2.0)
Generating win/loss arrays for group: (16.0, 5.0, 5.0, 3.0)
Generating win/loss arrays for group: (16.0, 5.0, 6.0, 1.0)
Generating win/loss arrays for group: (16.0, 5.0, 6.0, 2.0)
Generating win/loss arrays for group: (16.0, 5.0, 6.0, 3.0)
Generating win/loss arrays for group: (16.0, 5.0, 7.0, 1.0)
Generating win/loss arrays for group: (16.0, 5.0, 7.0, 2.0)
Generating win/loss arrays for group: (16.0, 5.0, 7.0, 3.0)
Generating win/loss arrays for group: (16.0, 5.0, 8.0, 1.0)
Generating win/loss arrays for group: (16.0, 5.0, 8.0, 2.0)
Generating win/loss arrays for group: (1

Generating win/loss arrays for group: (16.0, 7.0, 6.0, 3.0)
Generating win/loss arrays for group: (16.0, 7.0, 7.0, 1.0)
Generating win/loss arrays for group: (16.0, 7.0, 7.0, 2.0)
Generating win/loss arrays for group: (16.0, 7.0, 7.0, 3.0)
Generating win/loss arrays for group: (16.0, 7.0, 8.0, 1.0)
Generating win/loss arrays for group: (16.0, 7.0, 8.0, 2.0)
Generating win/loss arrays for group: (16.0, 7.0, 8.0, 3.0)
Generating win/loss arrays for group: (16.0, 7.0, 9.0, 1.0)
Generating win/loss arrays for group: (16.0, 7.0, 9.0, 2.0)
Generating win/loss arrays for group: (16.0, 7.0, 9.0, 3.0)
Generating win/loss arrays for group: (16.0, 7.0, 10.0, 1.0)
Generating win/loss arrays for group: (16.0, 7.0, 10.0, 2.0)
Generating win/loss arrays for group: (16.0, 7.0, 10.0, 3.0)
Generating win/loss arrays for group: (16.0, 7.0, 11.0, 1.0)
Generating win/loss arrays for group: (16.0, 7.0, 11.0, 2.0)
Generating win/loss arrays for group: (16.0, 7.0, 11.0, 3.0)
Generating win/loss arrays for gro

Generating win/loss arrays for group: (16.0, 9.0, 11.0, 1.0)
Generating win/loss arrays for group: (16.0, 9.0, 11.0, 2.0)
Generating win/loss arrays for group: (16.0, 9.0, 11.0, 3.0)
Generating win/loss arrays for group: (16.0, 9.0, 12.0, 1.0)
Generating win/loss arrays for group: (16.0, 9.0, 12.0, 2.0)
Generating win/loss arrays for group: (16.0, 9.0, 12.0, 3.0)
Generating win/loss arrays for group: (16.0, 9.0, 13.0, 1.0)
Generating win/loss arrays for group: (16.0, 9.0, 13.0, 2.0)
Generating win/loss arrays for group: (16.0, 9.0, 13.0, 3.0)
Generating win/loss arrays for group: (16.0, 9.0, 14.0, 1.0)
Generating win/loss arrays for group: (16.0, 9.0, 14.0, 2.0)
Generating win/loss arrays for group: (16.0, 9.0, 14.0, 3.0)
Generating win/loss arrays for group: (16.0, 9.0, 15.0, 1.0)
Generating win/loss arrays for group: (16.0, 9.0, 15.0, 2.0)
Generating win/loss arrays for group: (16.0, 9.0, 15.0, 3.0)
Generating win/loss arrays for group: (16.0, 9.0, 16.0, 1.0)
Generating win/loss arra

,index,date,game_id,game_name,group_id,location,player,points,position,score,season_id,week_id,wins_array,losses_array
0,0,"January 21, 2019",1.0,Attack From Mars,1.0,Dojo,Mark Lawrence,5.0,2.0,"3,220,471,780",16.0,1.0,"[Pete Hendricks, Peace Nick Mann, Sandy Graver]",[]
1,1,"January 21, 2019",1.0,Attack From Mars,1.0,Dojo,Pete Hendricks,3.0,1.0,"2,360,571,610",16.0,1.0,"[Peace Nick Mann, Sandy Graver]",[Mark Lawrence]
2,2,"January 21, 2019",1.0,Attack From Mars,1.0,Dojo,Peace Nick Mann,2.0,0.0,"1,388,813,900",16.0,1.0,[Sandy Graver],"[Mark Lawrence, Pete Hendricks]"
3,3,"January 21, 2019",1.0,Attack From Mars,1.0,Dojo,Sandy Graver,1.0,3.0,"590,859,230",16.0,1.0,[],"[Mark Lawrence, Pete Hendricks, Peace Nick Mann]"
0,721,"January 21, 2019",2.0,Twilight Zone,1.0,Dojo,Pete Hendricks,5.0,1.0,"491,827,690",16.0,1.0,"[Peace Nick Mann, Sandy Graver, Mark Lawrence]",[]
1,722,"January 21, 2019",2.0,Twilight Zone,1.0,Dojo,Peace Nick Mann,3.0,0.0,"70,556,260",16.0,1.0,"[Sandy Graver, Mark Lawrence]",[Pete Hendricks]
2,723,"January 21, 2019",2.0,Twilight Zone,1.0,Dojo,Sandy Graver,2.0,3.0,"58,630,840",16.0,1.0,[Mark Lawrence],"[Pete Hendricks, Peace Nick Mann]"
3,724,"January 21, 2019",2.0,Twilight Zone,1.0,Dojo,Mark Lawrence,1.0,2.0,"25,915,420",16.0,1.0,[],"[Pete Hendricks, Peace Nick Mann, Sandy Graver]"
0,1442,"January 21, 2019",3.0,Paragon,1.0,Dojo,Pete Hendricks,5.0,1.0,"168,050",16.0,1.0,"[Peace Nick Mann, Sandy Graver, Mark Lawrence]",[]
1,1443,"January 21, 2019",3.0,Paragon,1.0,Dojo,Peace Nick Mann,3.0,0.0,"137,570",16.0,1.0,"[Sandy Graver, Mark Lawrence]",[Pete Hendricks]


In [19]:
def concat_wins(df):
    return np.concatenate(df['wins_array'].values)

def concat_losses(df):
    return np.concatenate(df['losses_array'].values)

def total_stuff(series):
    series['wins'] = len(series['wins_array'])
    series['losses'] = len(series['losses_array'])
    return series

def wp(series):
    series['wp'] = float(series['wins']) / (series['wins'] + series['losses'])
    return series

def owp(series):
    print 'Calculating owp for {}, season {}'.format(series.name[1], series.name[0])
    opponents = np.concatenate([series['wins_array'], series['losses_array']])
    owp_total = 0
    for opponent in opponents:
        o_wins = df3.loc[(series.name[0], opponent)]['wins_array']
        o_losses = df3.loc[(series.name[0], opponent)]['losses_array']
        o_wins = [y for y in o_wins if y != series.name[1]]
        o_losses = [y for y in o_losses if y != series.name[1]]
        owp_total += ( float(len(o_wins)) / (len(o_wins) + len(o_losses)) )
    series['owp'] = owp_total / len(opponents)
    return series

def oowp(series):
    print 'Calculating oowp for {}, season {}'.format(series.name[1], series.name[0])
    opponents = np.concatenate([series['wins_array'], series['losses_array']])
    oowp_total = 0
    for opponent in opponents:
        oowp_total += df3.loc[(series.name[0], opponent)]['owp']
    series['oowp'] = oowp_total / len(opponents)
    return series

def sos(series):
    series['sos'] = (2 * series['owp'] + series['oowp']) / 3
    return series

def rpi(series):
    series['rpi'] = (.25 * series['wp']) + (.5 * series['owp']) + (.25 * series['oowp'])
    return series

grouped = df2.groupby(['season_id', 'player'])
# for name, group in grouped:
#     print name
#     print group
    
# records = grouped.sum().drop(['index', 'game_id', 'group_id', 'points', 'position', 'week_id'], axis=1)
print 'Applying function: {}'.format('concat_wins')
win_series = grouped.apply(concat_wins)
print 'Applying function: {}'.format('concat_losses')
loss_series = grouped.apply(concat_losses)
df3 = pd.concat(dict(wins_array = win_series, losses_array = loss_series), axis=1)
print 'Applying function: {}'.format('total_stuff')
df3 = df3.apply(total_stuff, axis=1)
print 'Applying function: {}'.format('wp')
df3 = df3.apply(wp, axis=1)
print 'Applying function: {}'.format('owp')
df3 = df3.apply(owp, axis=1)
print 'Applying function: {}'.format('oowp')
df3 = df3.apply(oowp, axis=1)
print 'Applying function: {}'.format('sos')
df3 = df3.apply(sos, axis=1)
print 'Applying function: {}'.format('rpi')
df3 = df3.apply(rpi, axis=1)
print df3
df3.loc[(16, 'Jared Schmidt')]

Applying function: concat_wins
Applying function: concat_losses
Applying function: total_stuff
Applying function: wp
Applying function: owp
Calculating owp for Aaron Crittenden, season 16.0
Calculating owp for Al Thomka, season 16.0
Calculating owp for Aleksander Kaczmarczyk, season 16.0
Calculating owp for Alex Fruzynski, season 16.0
Calculating owp for Allison Thrower, season 16.0
Calculating owp for Amy L. Covell-Murthy, season 16.0
Calculating owp for Amy Smith, season 16.0
Calculating owp for Andy Piccuta, season 16.0
Calculating owp for Angie Marinelli, season 16.0
Calculating owp for Barry Applbaum, season 16.0
Calculating owp for Bill Danylo, season 16.0
Calculating owp for Bob Steele, season 16.0
Calculating owp for Brandon Dye, season 16.0
Calculating owp for Brennan Ehrman, season 16.0
Calculating owp for Brett Berkman, season 16.0
Calculating owp for Brian Bower, season 16.0
Calculating owp for Brian Mendelssohn, season 16.0
Calculating owp for Brian Uplinger, season 16.0
C

Calculating oowp for Jerry Smith, season 16.0
Calculating oowp for Jesse Soracco, season 16.0
Calculating oowp for John Alioto, season 16.0
Calculating oowp for John Rogler, season 16.0
Calculating oowp for Jon Replogle, season 16.0
Calculating oowp for Jordan Retterer, season 16.0
Calculating oowp for Joshua Burfield, season 16.0
Calculating oowp for Kenny Parker, season 16.0
Calculating oowp for Kevin Brown, season 16.0
Calculating oowp for Kevin Vitale, season 16.0
Calculating oowp for Kyle Mueller, season 16.0
Calculating oowp for Lee Klevens, season 16.0
Calculating oowp for Lewis Bevans, season 16.0
Calculating oowp for Lindsey West, season 16.0
Calculating oowp for Lucas Herter, season 16.0
Calculating oowp for Lyle Applbaum, season 16.0
Calculating oowp for Mahesh Murthy, season 16.0
Calculating oowp for Mara Kline, season 16.0
Calculating oowp for Marco Leal, season 16.0
Calculating oowp for Marisa Bailey, season 16.0
Calculating oowp for Mark Lawrence, season 16.0
Calculating

KeyError: u'the label [14] is not in the [index]'

In [20]:
df3.loc[(16, 'Jared Schmidt')]

losses_array    [Lewis Bevans, Amy L. Covell-Murthy, Peace Nic...
wins_array      [Harold Lessure, Lewis Bevans, Amy L. Covell-M...
wins                                                           45
losses                                                         21
wp                                                       0.681818
owp                                                      0.521139
oowp                                                     0.513879
sos                                                      0.518719
rpi                                                      0.559494
Name: (16.0, Jared Schmidt), dtype: object

In [21]:
df4 = df3.drop(['wins_array', 'losses_array'], axis=1)
df4.reset_index(inplace=True)
stats_json = df4.to_json(orient='records')

# stats = db['stats']
# stats.insert_many(json.loads(stats_json))

json.loads(stats_json)

[{u'losses': 35,
  u'oowp': 0.4997315208,
  u'owp': 0.5087868854,
  u'player': u'Aaron Crittenden',
  u'rpi': 0.5077985451,
  u'season_id': 16.0,
  u'sos': 0.5057684305,
  u'wins': 37,
  u'wp': 0.5138888889},
 {u'losses': 21,
  u'oowp': 0.5030342921,
  u'owp': 0.5344397143,
  u'player': u'Al Thomka',
  u'rpi': 0.5596450968,
  u'season_id': 16.0,
  u'sos': 0.5239712402,
  u'wins': 42,
  u'wp': 0.6666666667},
 {u'losses': 6,
  u'oowp': 0.5190788405,
  u'owp': 0.5655932848,
  u'player': u'Aleksander Kaczmarczyk',
  u'rpi': 0.636250563,
  u'season_id': 16.0,
  u'sos': 0.55008847,
  u'wins': 51,
  u'wp': 0.8947368421},
 {u'losses': 37,
  u'oowp': 0.4906495674,
  u'owp': 0.499596099,
  u'player': u'Alex Fruzynski',
  u'rpi': 0.4884024703,
  u'season_id': 16.0,
  u'sos': 0.4966139218,
  u'wins': 32,
  u'wp': 0.4637681159},
 {u'losses': 37,
  u'oowp': 0.4843029959,
  u'owp': 0.5064227963,
  u'player': u'Allison Thrower',
  u'rpi': 0.4701204804,
  u'season_id': 16.0,
  u'sos': 0.4990495295,
  u

In [45]:
season_grouped = df4.groupby('season_id')
df5 = pd.DataFrame()
for name, group in season_grouped:
    group.sort_values('wp', ascending=False, inplace=True)
    group['wp_rank'] = group.reset_index().index + 1
    group.sort_values('owp', ascending=False, inplace=True)
    group['owp_rank'] = group.reset_index().index + 1
    group.sort_values('oowp', ascending=False, inplace=True)
    group['oowp_rank'] = group.reset_index().index + 1
    group.sort_values('sos', ascending=False, inplace=True)
    group['sos_rank'] = group.reset_index().index + 1
    group.sort_values('rpi', ascending=False, inplace=True)
    group['rpi_rank'] = group.reset_index().index + 1
    df5 = df5.append(group)
    
df5.head(20)

,season_id,player,wins,losses,wp,owp,oowp,sos,rpi,wp_rank,owp_rank,oowp_rank,sos_rank,rpi_rank
2,16.0,Aleksander Kaczmarczyk,51,6,0.894737,0.565593,0.519079,0.550088,0.636251,1,3,6,4,1
91,16.0,Pete Hendricks,52,11,0.825397,0.491110,0.512067,0.498095,0.579921,5,71,15,58,2
39,16.0,Greg DeFeo,61,11,0.847222,0.479007,0.503850,0.487288,0.577271,2,86,37,83,3
62,16.0,Jon Replogle,46,20,0.696970,0.549928,0.509890,0.536582,0.576679,14,8,20,7,4
25,16.0,Cryss Stephens,40,8,0.833333,0.476997,0.509328,0.487774,0.574164,4,89,23,81,5
70,16.0,Lewis Bevans,47,16,0.746032,0.516255,0.503074,0.511862,0.570404,8,31,44,31,6
30,16.0,Dave Lancaster,45,18,0.714286,0.514142,0.520022,0.516102,0.565648,10,37,5,24,7
16,16.0,Brian Mendelssohn,15,3,0.833333,0.459086,0.491545,0.469906,0.560763,3,102,93,102,8
105,16.0,Todd Williams,51,21,0.708333,0.513923,0.504050,0.510632,0.560058,12,38,36,33,9
1,16.0,Al Thomka,42,21,0.666667,0.534440,0.503034,0.523971,0.559645,18,16,45,16,10


In [36]:
df5 = df4.sort_values('wp', ascending=False)
df5['wp_rank'] = df5.reset_index().index + 1
df5.sort_values('owp', ascending=False, inplace=True)
df5['owp_rank'] = df5.reset_index().index + 1
df5.sort_values('oowp', ascending=False, inplace=True)
df5['oowp_rank'] = df5.reset_index().index + 1
df5.sort_values('sos', ascending=False, inplace=True)
df5['sos_rank'] = df5.reset_index().index + 1
df5.sort_values('rpi', ascending=False, inplace=True)
df5['rpi_rank'] = df5.reset_index().index + 1
df5.head(10)

,season_id,player,wins,losses,wp,owp,oowp,sos,rpi,wp_rank,owp_rank,oowp_rank,sos_rank,rpi_rank
2,16.0,Aleksander Kaczmarczyk,51,6,0.894737,0.565593,0.519079,0.550088,0.636251,1,3,6,4,1
91,16.0,Pete Hendricks,52,11,0.825397,0.491110,0.512067,0.498095,0.579921,5,71,15,58,2
39,16.0,Greg DeFeo,61,11,0.847222,0.479007,0.503850,0.487288,0.577271,2,86,37,83,3
62,16.0,Jon Replogle,46,20,0.696970,0.549928,0.509890,0.536582,0.576679,14,8,20,7,4
25,16.0,Cryss Stephens,40,8,0.833333,0.476997,0.509328,0.487774,0.574164,4,89,23,81,5
70,16.0,Lewis Bevans,47,16,0.746032,0.516255,0.503074,0.511862,0.570404,8,31,44,31,6
30,16.0,Dave Lancaster,45,18,0.714286,0.514142,0.520022,0.516102,0.565648,10,37,5,24,7
16,16.0,Brian Mendelssohn,15,3,0.833333,0.459086,0.491545,0.469906,0.560763,3,102,93,102,8
105,16.0,Todd Williams,51,21,0.708333,0.513923,0.504050,0.510632,0.560058,12,38,36,33,9
1,16.0,Al Thomka,42,21,0.666667,0.534440,0.503034,0.523971,0.559645,18,16,45,16,10


In [37]:
stats_with_rank_json = df5.to_json(orient='records')
stats = db['stats_rank']
stats.insert_many(json.loads(stats_with_rank_json))

json.loads(stats_with_rank_json)

[{u'losses': 6,
  u'oowp': 0.5190788405,
  u'oowp_rank': 6,
  u'owp': 0.5655932848,
  u'owp_rank': 3,
  u'player': u'Aleksander Kaczmarczyk',
  u'rpi': 0.636250563,
  u'rpi_rank': 1,
  u'season_id': 16.0,
  u'sos': 0.55008847,
  u'sos_rank': 4,
  u'wins': 51,
  u'wp': 0.8947368421,
  u'wp_rank': 1},
 {u'losses': 11,
  u'oowp': 0.5120666932,
  u'oowp_rank': 15,
  u'owp': 0.4911098638,
  u'owp_rank': 71,
  u'player': u'Pete Hendricks',
  u'rpi': 0.5799208115,
  u'rpi_rank': 2,
  u'season_id': 16.0,
  u'sos': 0.4980954736,
  u'sos_rank': 58,
  u'wins': 52,
  u'wp': 0.8253968254,
  u'wp_rank': 5},
 {u'losses': 11,
  u'oowp': 0.50385005,
  u'oowp_rank': 37,
  u'owp': 0.4790066656,
  u'owp_rank': 86,
  u'player': u'Greg DeFeo',
  u'rpi': 0.5772714008,
  u'rpi_rank': 3,
  u'season_id': 16.0,
  u'sos': 0.4872877937,
  u'sos_rank': 83,
  u'wins': 61,
  u'wp': 0.8472222222,
  u'wp_rank': 2},
 {u'losses': 20,
  u'oowp': 0.5098904259,
  u'oowp_rank': 20,
  u'owp': 0.5499275621,
  u'owp_rank': 8,
 